In [1]:
import ray
import pickle
import sys
import numpy as np
import math
import os
#my module
from LDPC_code import LDPC_construction
from LDPC_code import LDPC_encode
from LDPC_code import LDPC_decode
from polar_code import polar_construction
from polar_code import polar_encode
from polar_code import polar_decode
from polar_code import RCA
from polar_code import iGA
from polar_code import monte_carlo_construction
from turbo_code import turbo_construction
from turbo_code import turbo_encode
from turbo_code import turbo_decode
from modulation import modulation
from modulation.BICM import make_BICM
from channel import AWGN

In [2]:
class Mysystem_LDPC():
    def __init__(self,M,K):
        self.M=M
        self.K=K
        #self.N=self.K*int(np.log2(self.M))
        self.N=self.K*2
        self.BICM=True 
        self.BICM_ID=True
        
        if self.BICM_ID==True:
            self.BICM=True
            self.BICM_ID_itr=10
                
        #coding
        self.cd=LDPC_construction.coding(self.N,self.K)
        self.ec=LDPC_encode.encoding(self.cd)
        self.dc=LDPC_decode.decoding(self.cd)
        #modulation
        self.modem=modulation.QAMModem(self.M)
        

        #channel
        self.ch=AWGN._AWGN()
        
        #filename
        self.filename="LDPC_code_{}_{}_{}".format(self.N,self.K,self.M)
        if self.BICM==True:
            self.BICM_int,self.BICM_deint=make_BICM(self.N,self.M)
            self.filename=self.filename+"_BICM"
        
        #output filename to confirm which program I run
        print(self.filename)
    
    def main_func(self,EsNodB):
        EsNo = 10 ** (EsNodB / 10)
        No=1/EsNo

        info,cwd=self.ec.LDPC_encode()
        info=cwd #BICMのとき、cwdがインターリーブされてしまい、比較できなくなる為、infoをcwdに変更する
        if self.BICM==True:
            cwd=cwd[self.BICM_int]
        TX_conste=self.modem.modulate(cwd)
        
        #channel
        RX_conste=self.ch.add_AWGN(TX_conste,No)
        
        #at the reciever
        if self.BICM_ID==False:
            Lc=self.modem.demodulate(RX_conste,No)
            if self.BICM==True:
                Lc=Lc[self.BICM_deint]
            EST_cwd,_=self.dc.LDPC_decode(Lc)
            return info,EST_cwd
        
        
        elif self.BICM_ID==True:
            
            #demodulate      
            Lc,[zeros,ones]=self.modem.demodulate(RX_conste,No,self.BICM_ID)
            
            ###check
            num=self.modem.calc_exp(zeros,No)
            denum=self.modem.calc_exp(ones,No)
            Lc_check=(np.transpose(num[0]) - np.transpose(denum[0])).ravel(order='F')
            #print(Lc3)
            if np.any(Lc!=Lc_check):
                print("Lc is different")
            ###check end
            
            return Lc,[zeros,ones]

In [3]:
def key_preparation(self):
    """ Creates the coordinates
    where either zeros or ones can be placed in the signal constellation..
    Returns
    -------
    zeros : list of lists of complex values
        The coordinates where zeros can be placed in the signal constellation.
    ones : list of lists of complex values
        The coordinates where ones can be placed in the signal constellation.
    """

    zeros = [[] for i in range(self.N)]
    ones = [[] for i in range(self.N)]

    bin_seq = self.de2bin(self.m)

    for bin_idx, bin_symb in enumerate(bin_seq):
        if self.bin_input == True:
            key = bin_symb
        else:
            key = bin_idx
        for possition, digit in enumerate(bin_symb):
            if digit == '0':
                zeros[possition].append(key)
            else:
                ones[possition].append(key)
    
    #from str list to int array 
    for i in range(len(zeros)):
        zeros[i]=np.array([int(s, 2) for s in zeros[i]])
        ones[i]=np.array([int(s, 2) for s in ones[i]])
                
    return zeros, ones

In [78]:
K=512
M=4
EsNodB=0
EsNo = 10 ** (EsNodB / 10)
No=1/EsNo
mysys=Mysystem_LDPC(M,K)
modem=mysys.modem
modem.code_book

(2, 4)
LDPC_code_1024_512_4_BICM


{'00': (-0.7071067811865475+0.7071067811865475j),
 '01': (-0.7071067811865475-0.7071067811865475j),
 '11': (0.7071067811865475-0.7071067811865475j),
 '10': (0.7071067811865475+0.7071067811865475j)}

In [79]:
np.arange(1024)

array([   0,    1,    2, ..., 1021, 1022, 1023])

In [80]:
#prepere matrix
#codebookから、シンボルの生起確率を計算するためのマトリクスを計算する

tmp=modem.code_book
mat=np.zeros((len(tmp),int(np.log2(len(tmp)))),dtype='int')
for i,bi in enumerate(tmp):
    tmp=list(bi)
    #print(tmp)
    for j in range(len(tmp)):
        mat[i,j]=int(tmp[j])
    #print(mat[i])
#mat=-1*(2*mat-1) #[0,1] to[1,-1]

In [81]:
print(mat)

for i i

[[0 0]
 [0 1]
 [1 1]
 [1 0]]


In [82]:
zeros_key,ones_key=key_preparation(mysys.modem)
print(modem.code_book)
print(zeros_key)
print(ones_key)

{'00': (-0.7071067811865475+0.7071067811865475j), '01': (-0.7071067811865475-0.7071067811865475j), '11': (0.7071067811865475-0.7071067811865475j), '10': (0.7071067811865475+0.7071067811865475j)}
[array([0, 1]), array([0, 2])]
[array([2, 3]), array([1, 3])]


In [83]:
'''
def llr2prob(llr):
    log=True
    p0=math.exp(llr)/(1+math.exp(llr))
    p1=1/(1+math.exp(llr))
    if log==True:
        p0=math.log(p0)
        p1=math.log(p1)
    return np.array([p0,p1])
'''

'\ndef llr2prob(llr):\n    log=True\n    p0=math.exp(llr)/(1+math.exp(llr))\n    p1=1/(1+math.exp(llr))\n    if log==True:\n        p0=math.log(p0)\n        p1=math.log(p1)\n    return np.array([p0,p1])\n'

In [84]:
#test
tmp=np.arange(10).reshape(2,5)
print(tmp)
index=np.array([[1,1],[1,1]])
tmp[index]

[[0 1 2 3 4]
 [5 6 7 8 9]]


array([[[5, 6, 7, 8, 9],
        [5, 6, 7, 8, 9]],

       [[5, 6, 7, 8, 9],
        [5, 6, 7, 8, 9]]])

In [194]:
Lc,[zeros,ones]=mysys.main_func(EsNodB) #Lcはデインターリーブされていない
#decode 
EST_cwd,EX_info=mysys.dc.LDPC_decode(Lc[mysys.BICM_deint]) #MAPデコーダで出てきた外部値を取得

Pre_info=EX_info[mysys.BICM_int]#順番の入れ替えをして、事前値にする

#lambda_Lc=EX_info+Lc #更新されたLLRの値

symbol_num=int(len(Lc)/modem.N) #シンボルの長さ
print(symbol_num)

#print(Pre_info[:30])
#Pre_info=Pre_info.reshape([modem.N,symbol_num],order='F') #各シンボルで受信したビットごとに並べ替える　(symbol_num*bits_in_symbol)

Pre_info_mat=mat@(Pre_info.reshape([modem.N,symbol_num],order='F'))#+Lc.reshape([modem.N,symbol_num],order='F'))

print(Pre_info_mat.shape)
print(zeros_key)
print(zeros.shape)

print(modem.N)

ex_mat_z=np.zeros(zeros.shape)
ex_mat_o=np.zeros(zeros.shape)

for i in range(symbol_num): #1シンボル毎
    for j in range(modem.N): #シンボル内の1ビットごと
        ex_mat_z[:,i,j]-=Pre_info_mat[zeros_key[j],i]
        ex_mat_o[:,i,j]-=Pre_info_mat[ones_key[j],i]

#print(np.sum(ex_mat_z,axis=0,keepdims=True)-np.sum(ex_mat_o,axis=0,keepdims=True))


c=np.exp(ex_mat_z)
#c=np.prod(c,axis=2,keepdims=True)/c
#c=np.clip(c,10**(-15),10**15)
if np.any(np.isnan(c))==True:
       print("c err") 

d=np.exp(ex_mat_o)
#d=np.prod(d,axis=2,keepdims=True)/d
#d=np.clip(d,10**(-15),10**15)
if np.any(np.isnan(d))==True:
       print("d err") 

a=np.exp(-1*np.array(zeros)/No)
a=a*c
a=np.sum(a,axis=0,keepdims=True)
#a=np.clip(a,10**(-15),10**15)
a=np.log(a)
if np.any(np.isnan(a))==True:
   print("a err") 

b=np.exp(-1*np.array(ones)/No)
b=b*d
b=np.sum(b,axis=0,keepdims=True)
#b=np.clip(b,10**(-15),10**15)
b=np.log(b)
if np.any(np.isnan(b))==True:
       print("b err") 
#a=modem.calc_exp(zeros,No)
#b=modem.calc_exp(ones,No)

Lc4=(np.transpose(a[0]) - np.transpose(b[0])).ravel(order='F')   
#print(Pre_info[:30])
print(Lc)
print(Lc4-Pre_info)

512
(4, 512)
[array([0, 1]), array([0, 2])]
(2, 512, 2)
2
[ 4.93072518  0.65904768 -3.87627123 ...  2.07553647 -0.45334692
 -2.0851421 ]
[  5.56217002   0.65904369  -0.70826416 ... -14.71350054   1.6004103
   5.17954694]


In [251]:
Lc,[zeros,ones]=mysys.main_func(EsNodB) #Lcはデインターリーブされていない
#decode 
EST_cwd,EX_info=mysys.dc.LDPC_decode(Lc[mysys.BICM_deint]) #MAPデコーダで出てきた外部値を取得

Pre_info=EX_info[mysys.BICM_int]#順番の入れ替えをして、事前値にする

#lambda_Lc=EX_info+Lc #更新されたLLRの値

symbol_num=int(len(Lc)/modem.N) #シンボルの長さ
print(symbol_num)

Pre_info=Pre_info.reshape(symbol_num,modem.N) #各シンボルで受信したビットごとに並べ替える　(symbol_num*bits_in_symbol)

delta_St=np.zeros((M,symbol_num)) #各信号点の生起確率を計算する
for i in range(symbol_num): 
    delta_St[:,i]=-1*mat@Pre_info[i,:] #i番目のシンボルにおいて、それぞれのシンボルの正規確率の差分を求める

llr2prob(EX_info)

print(mat)
print(Pre_info[0,:])
print(delta_St[:,0])

512


TypeError: only size-1 arrays can be converted to Python scalars

In [233]:
print(delta_St)

[[  0.           0.           0.         ...   0.           0.
    0.        ]
 [ -9.24119333  -9.02917316   9.8219848  ...  -4.77631019  -3.08403718
    7.42137948]
 [-13.66339525 -12.24321817  21.69475357 ... -16.22463737   5.93417585
   -3.26593909]
 [ -4.42220192  -3.21404501  11.87276878 ... -11.44832718   9.01821303
  -10.68731858]]


In [234]:
a=modem.calc_exp(zeros,No)
b=modem.calc_exp(ones,No)
print(a.shape)
print(a.ravel().shape)
Lc3=(np.transpose(a[0]) - np.transpose(b[0])).ravel(order='F')
if np.any(Lc3!=Lc):
    print("error!")

(1, 512, 2)
(1024,)


In [224]:
LLR = []
for i in range(modem.N):#すべてのシンボルのiビット目のビットについて考える
    new_zeros=zeros[:,:,i]+delta_St[zeros_key[i],:]
    new_ones=ones[:,:,i]+delta_St[ones_key[i],:]
    
    num_post=modem.calc_exp(new_zeros,No)
    denum_post=modem.calc_exp(new_ones,No)
    llr = np.transpose(num_post[0]) - np.transpose(denum_post[0]) #二次元配列になってしまっているので、1次元に直す
    LLR.append(llr)

result = np.zeros((symbol_num * len(zeros_key)))
for i, llr_i in enumerate(LLR):
    result[i::len(zeros_key)] = llr_i

print(result.shape)

(1024,)


In [237]:


print(result-Lc-Pre_info.ravel())

[-12.24606445 -18.44567462  -5.03469357 ...  -3.68426571  -9.81412753
   5.50035485]
